In [1]:
import numpy as np
import pandas as pd
import snsims
print(snsims.__version__)

Some imports failed, which implies some dependencies are missing as described below
No module named 'mpl_toolkits.basemap'
Visulization functions based on maps will not work
1.2.12


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [3]:
from astropy.cosmology import Planck15 as cosmo
from astropy.cosmology import FlatLambdaCDM

In [4]:
import sncosmo

In [5]:
DC2 = FlatLambdaCDM(H0=71, Om0=0.265, Ob0=0.0448)

In [6]:
#GCRCatalogs.get_available_catalogs(include_default_only=False)
#gc.cosmology

In [7]:
import GCRCatalogs

In [8]:
gc = GCRCatalogs.load_catalog('cosmoDC2_v0.1_test')
#gc = GCRCatalogs.load_catalog('cosmoDC2_v0.1')

In [9]:
gals = pd.DataFrame(gc.get_quantities(['galaxy_id','morphology/diskHalfLightRadiusArcsec','size_disk_true', 'size_bulge_true', 'diskMassStellar', 
                                       'totalMassStellar', 'ra', 'dec','redshift', 'morphology/diskMinorAxisArcsec', 
                                       'morphology/diskMinorAxisArcsec', 'morphology/spheroidMinorAxisArcsec',
                                       'morphology/spheroidMinorAxisArcsec', 'morphology/positionAngle', 
                                       'morphology/spheroidHalfLightRadiusArcsec','position_x','position_y','position_z']))

In [13]:
xx = np.unique(gals['galaxy_id'])
pp = np.where(gals['galaxy_id'] == xx[0])

In [29]:
gals.head()

,dec,diskMassStellar,galaxy_id,morphology/diskHalfLightRadiusArcsec,morphology/diskMinorAxisArcsec,morphology/positionAngle,morphology/spheroidHalfLightRadiusArcsec,morphology/spheroidMinorAxisArcsec,position_x,position_y,position_z,ra,redshift,size_bulge_true,size_disk_true,totalMassStellar
0,-30.135204,1.583015e+09,0,9.919187,9.881069,3.894788,6.460957,6.349892,33.372276,44.261086,-32.178566,52.984187,0.021644,6.460957,9.919188,1.793685e+10
1,-30.032378,4.997883e+07,1,2.233699,0.941040,98.332085,1.574325,1.482722,33.465843,44.264641,-32.079933,52.909258,0.021718,1.574325,2.233699,3.225152e+08
2,-30.040808,1.158037e+08,2,4.516936,1.441848,128.619049,3.009337,2.658643,33.259758,44.264366,-32.018967,53.079202,0.022176,3.009337,4.516936,3.336403e+09
3,-30.277000,4.270559e+08,3,3.198411,2.135988,6.036642,2.156365,2.043655,33.343285,44.211601,-32.328987,52.977314,0.021256,2.156365,3.198411,5.271975e+08
4,-34.256402,7.540549e+07,4,5.322864,4.223814,30.911007,3.069553,3.059789,28.199570,34.384472,-30.285257,50.643961,0.017618,3.069552,5.322864,1.113713e+08


In [25]:
gals.iloc[pp]

,dec,diskMassStellar,galaxy_id,morphology/diskHalfLightRadiusArcsec,morphology/diskMinorAxisArcsec,morphology/positionAngle,morphology/spheroidHalfLightRadiusArcsec,morphology/spheroidMinorAxisArcsec,position_x,position_y,position_z,ra,redshift,size_bulge_true,size_disk_true,totalMassStellar
0,-30.135204,1.583015e+09,0,9.919187,9.881069,3.894788,6.460957,6.349892,33.372276,44.261086,-32.178566,52.984187,0.021644,6.460957,9.919188,1.793685e+10
13548109,-26.951122,3.643373e+07,0,0.570043,0.151783,121.028328,0.408184,0.322267,1426.578125,1632.927734,-1102.481323,48.858493,1.041873,0.408184,0.570043,1.581905e+10
46878081,-31.295597,3.840556e+07,0,0.208302,0.176626,34.814911,0.131435,0.123699,2061.995605,2462.867188,-1952.645020,50.062792,2.019015,0.131435,0.208302,1.068898e+09


In [ ]:
#gc.list_all_quantities()

## SN hosted in the galaxies

In [10]:
import astropy.units as u
from astropy.cosmology import Planck13, z_at_value

#### Use the next part only if the redshifts are not correct
The redshift values were incorrect for the earlier versions of DC2 so I used comoving distances to get redshifts instead since comoving distances were fine - this issue should be resolved in the later versions of DC2

In [11]:
com_dist = np.sqrt((gals.position_x**2) + (gals.position_y**2)+(gals.position_z**2))

min_indx = np.where(com_dist == np.min(com_dist ))[0][0]
max_indx = np.where(com_dist == np.max(com_dist ))[0][0]

zmin = z_at_value(DC2.comoving_distance, com_dist[min_indx] * u.Mpc)
zmax = z_at_value(DC2.comoving_distance, com_dist[max_indx] * u.Mpc)

zgrid = np.logspace(np.log10(zmin), np.log10(zmax), 50)
cosmology = DC2
CDgrid = cosmology.comoving_distance(zgrid)*DC2.H0/100.
#  use interpolation to get redshifts for satellites only
new_redshifts = np.interp(com_dist, CDgrid, zgrid)

gals.redshift = new_redshifts

#### Plotting redshift distribution

In [ ]:
fig, ax = plt.subplots()
_ = ax.hist(gals.redshift, bins=np.arange(0., 1.01, 0.01), histtype='step')

In [12]:
zdist = snsims.PowerLawRates(rng=np.random.RandomState(1),
                             alpha=2.6e-5,
                             fieldArea=54,
                             surveyDuration=10.,
                             cosmo=DC2,
                             zbinEdges=np.arange(0.001, 1.4, 0.02))

In [13]:
SNPop = snsims.GMM_SALT2Params(numSN=None, zSamples=zdist.zSamples, rng=np.random.RandomState(2),
                               mjdmin=59580, cosmo=DC2, surveyDuration=10.)

In [14]:
MainSN = SNPop.paramSamples

In [15]:
MainSN['random_Hosting'] = np.random.uniform(size=len(MainSN))

In [16]:
hostedSN = MainSN.query('z < 1.0 and random_Hosting < 0.9')

In [ ]:
fig, ax = plt.subplots()
hostedSN.z.hist(histtype='step', ax=ax)
MainSN.z.hist(histtype='step', ax=ax, color='r')

In [ ]:
hostedSN.x0.describe()

In [ ]:
hostedSN.query('snid ==3')

In [17]:
redshiftvarx = 'redshift'
zmaxx = 0.96
query = '{0} < {1}'.format('redshift', zmaxx)
gals.query(query)
print(query)

redshift < 0.96


In [18]:
from snsims import SersicSamples

In [19]:
class HostingSN(object):
    """ Class to associate SN in the SN params table with hosts
    
    Parameters
    ----------
    
    """
    def __init__(self,
                 galsdf,
                 snpops,
                 redshiftvar='redshift',
                 rng=np.random.RandomState(23),
                 assignBinWidth=0.02,
                 zmax=0.96,
                 hostprob=0.9):
        self.arcsec2Deg = 1.0 / 3600.
        self.snparams = snpops.paramSamples
        self.rng = rng
        self.zmax = zmax
        self.hostprob = hostprob
        self.redshiftvar = 'redshift'
        querystr = '{0} < {1}'.format(redshiftvar, zmax)
        self.galsdf = galsdf.query(querystr)
        
        # Sample of SN split into hosted/hostless
        hstd, hstless = self.splitByHosting(self.snparams, rng=self.rng,
                                            hostprob=self.hostprob, zmax=self.zmax)
        self.hostedSN = hstd
        self.hostlessSN = hstless
    
        self.ss = SersicSamples(self.rng)
    
    @staticmethod
    def splitByHosting(snparams, rng, hostprob=0.9, zmax=0.96):
        """Split the SN sample by hosting type
        """
        #snparams= pops.paramSamples
        snparams['rand_host'] = rng.uniform(size=len(snparams))
        hosted = snparams.query('z < @zmax and rand_host < @hostprob')
        hostlessSN = snparams.query('z >= @zmax or rand_host >= @hostprob')
        return hosted, hostlessSN
    
    def assignHosts(self, binwidth=0.02):
        """
        Find hosts in redshift bins of 0.02 and populate them weighting
        them by stellar mass of host galaxies.
        
        Parameters
        ----------
        binwidth : float, defaults to 0.02
            width of redshift bin
        """
        galsdf = self.galsdf
        self.hostedSN['zbin'] = self.hostedSN.z // binwidth
        self.hostedSN['zbin'] = self.hostedSN.zbin.astype(np.int)
        galsdf['zbin'] = galsdf.redshift // binwidth
        galsdf['zbin'] = galsdf.zbin.astype(np.int)
        
        
        
        syslist = []

        for zbin in self.hostedSN.zbin.unique():
            querystring = 'zbin==@zbin'
            hostedtmp = self.hostedSN.query(querystring)
            numSN = hostedtmp.z.size
            gtmp  = galsdf.query(querystring)
            tot = gtmp.totalMassStellar.sum()
            #return gtmp
            #print (zbin, gtmp.totalMassStellar.sum(), tot, numSN
            p = gtmp.totalMassStellar/tot
            gids = np.random.choice(gtmp.reset_index().galaxy_id, size=numSN, replace=False,
                                    p=p.values)
            #print(len(hostedtmp), len(gids))
            syslist.append(pd.DataFrame(dict(snid=hostedtmp.reset_index().snid, galaxy_id=gids)))
        
        joiner = pd.concat(syslist).set_index('snid')
        params = self.hostedSN.set_index('snid').join(joiner).set_index('galaxy_id').join(galsdf.set_index('galaxy_id'), rsuffix='_gals')
        #params = self.hostedSN.reset_index().set_index('snid').join(joiner).reset_index().set_index('galaxy_id').join(self.galsdf, rsuffix='_gals')
        params.rename(columns=dict(raJ2000='raJ2000_gal', decJ2000='decJ2000_gal', redshift='z'), inplace=True)
        k = joiner.reset_index().set_index('galaxy_id')
        print(k.columns)
        xx = params.join(k)
        cols = params.columns
        dropcols = ['index', 'z', 'zbin', 'zbin_gals', 'rand_host'] #+ list(a for a in cols if 'zbin' not in a)
        keepcols = list(col for col in cols if col not in dropcols)
        #xx = joiner.reset_index().set_index('galaxy_id').join(params)
        return joiner, xx#[keepcols]
        #return gtmp
    
    def write(self, paramsdf, fname, ftype='hdf'):
        if ftype == 'hdf':
            paramsdf.to_hdf(fname, key='0')
        return
    #def sampleAngles(df, a, b):
    #    return self.ss.sampleAngles(df[a], df[b])[0]
    @property
    def get_positions(self, paramsdf, rng, disk_a='size_disk_true', disk_b='morphology/diskMinorAxisArcsec',
                      bulge_a='size_disk_true', bulge_b='morphology/spheroidMinorAxisArcsec',
                      diskhalfLight='morphology/diskHalfLightRadiusArcsec',
                      bulgehalfLight='morphology/spheroidHalfLightRadiusArcsec',
                      positionAngle='morphology/positionAngle'):
        """
        Obtain ra dec of the SN and add it to the parameter table
        """
        paramsdf['rand_SM'] = rng.uniform(size=len(paramsdf))
        paramsdf['diskmassratio'] = paramsdf['diskMassStellar'] / paramsdf['totalMassStellar']
        paramsdf['inDisk'] = paramsdf['diskmassratio'] > paramsdf['rand_SM']
        paramsdf.inDisk = paramsdf.inDisk.astype(np.int)
        x = paramsdf['inDisk']
        # Add angle samples
        paramsdf['diskAngle'] = self.ss.sampleAngles(paramsdf[disk_a], paramsdf[disk_b], numSamples=len(paramsdf))
        paramsdf['bulgeAngle'] = self.ss.sampleAngles(paramsdf[bulge_a], paramsdf[bulge_b], numSamples=len(paramsdf))
        
        paramsdf['diskRadius'] = self.ss.sampleRadius(paramsdf[diskhalfLight], numSamples=len(paramsdf), sersicIndex=1)
        paramsdf['bulgeRadius'] = self.ss.sampleRadius(paramsdf[bulgehalfLight], numSamples=len(paramsdf), sersicIndex=4)
        paramsdf['theta'] = paramsdf['diskAngle'] * x + (1-x) *paramsdf['bulgeAngle'] - paramsdf[positionAngle] +90 
        paramsdf['theta'] = np.radians(paramsdf.theta)
        paramsdf['radial'] = paramsdf['diskRadius'] * x + paramsdf['bulgeRadius']* (1-x)
        paramsdf['deltaRA'] = np.cos(paramsdf['theta']) * paramsdf['radial']
        paramsdf['deltaDec'] = np.sin(paramsdf['theta']) * paramsdf['radial']
        paramsdf['snra'] = paramsdf['raJ2000_gal'] + paramsdf['deltaRA'] * self.arcsec2Deg
        paramsdf['sndec'] = paramsdf['decJ2000_gal'] +paramsdf['deltaDec'] * self.arcsec2Deg
        return paramsdf.set_index('snid')

In [20]:
hstsn = HostingSN(gals, SNPop)
x     = hstsn.assignHosts()

In [ ]:
#x[1].columns
#x[1].to_csv('Hosted_SN_table_cosmoDC2.csv', sep=',')

#### saving the files to be passed to OpSim_rows notebook

In [22]:
vaaaal01 = x[1].query('z<0.1')

In [23]:
vaaaal02 = x[1].query('z<0.2')

In [24]:
vaaaal03 = x[1].query('z<0.3')

In [25]:
vaaaal01.to_csv('Hosted_SN_table_cosmoDC2_test_z0.1.csv', sep=',')

In [26]:
vaaaal02.to_csv('Hosted_SN_table_cosmoDC2_test_z0.2.csv', sep=',')

In [27]:
vaaaal03.to_csv('Hosted_SN_table_cosmoDC2_test_z0.3.csv', sep=',')

#### Scratch work

In [ ]:
hstsn.arcsec2Deg

In [ ]:
t = hstsn.get_positions(x[1], rng = np.random.RandomState(2))

In [ ]:
t.deltaRA.hist(bins=np.arange(-25, 25, 0.1))

In [ ]:
x[0].reset_index().set_index('galaxy_id').join(x[1])

In [ ]:
x[1]['morphology/positionAngle'].hist()

In [ ]:
x[1].columns

In [ ]:
'snid' in x[0].reset_index().set_index('galaxy_id').columns

In [ ]:
'snid' in x[1].join(x[0].set_index('galaxy_id')).columns

In [ ]:
hstsn.get_positions(x[1], np.random.RandomState(1))

In [ ]:
gals.head()

In [ ]:
gals.query('galaxy_id ==19728327')

In [ ]:
x[1].head()

In [ ]:
hstsn.get_positions(x[1], rng=np.random.RandomState(1))

In [ ]:
rng.uniform(size=10)

In [ ]:
def get_positions(paramsdf, rng):
        paramsdf['rand_SM'] = rng.uniform(size=len(paramsdf))
        paramsdf['diskmassratio'] = paramsdf['diskMassStellar'] / paramsdf['totalMassStellar']
        paramsdf['inDisk'] = paramsdf['diskmassratio'] > paramsdf['rand_SM']
        paramsdf.inDisk = paramsdf.inDisk.astype(np.int)
        
        x = paramsdf['inDisk']
        #paramsdf['eff_a'] = paramsdf['size_disk_true'] * x  + params['size_bulge_true'] * (1 - x)
        #paramsdf['eff_b'] = paramsdf['morphology/diskMinorAxisArcsec'] * x +\
        #    paramsdf['morphology/spheroidMinorAxisArcsec'] * (1-x)
        return paramsdf.reset_index().set_index('snid')

In [ ]:
y = get_positions(hstsn.assignHosts()[1], np.random.RandomState(1))

In [ ]:
y.plot.pie(y='inDisk')

In [ ]:
x = hstsn.assignHosts()

In [ ]:
hstsn.get_positions(np.random.RandomState(0), x)

In [ ]:
rng = np.random.RandomState(0)

In [ ]:
rng.uniform

In [ ]:
x[1]#.rename(columns=dict(raJ2000='raJ2000_gal', decJ2000='decJ2000_gal'), inplace=True)

In [ ]:
hstsn.galsdf.totalMassStellar.apply(np.log10).hist(histtype='step', normed=1)
hstsn.assignHosts()[1].totalMassStellar.apply(np.log10).hist(histtype='step', normed=1)

In [ ]:
hstsn.hostedSN.z.size

In [ ]:
hstsn.galsdf.redshift.size

In [ ]:
hstsn.hostlessSN.z.size

In [ ]:
params = hstsn.assignHosts()

In [ ]:
params[1].columns

In [ ]:
joiner.head()

In [ ]:
len(joiner)

In [ ]:
hstsn.hostedSN.head()

In [ ]:
hstsn.galsdf.head()

In [ ]:
hstsn.hostedSN.reset_index().set_index('snid').join(joiner).set_index('galaxy_id').join(hstsn.galsdf, rsuffix='_gals').head()

In [ ]:
plt.hist(zdist.zSamples, bins=np.arange(0., 1.4, 0.04))

In [ ]:
from lsst.sims.utils import samplePatchOnSphere

In [ ]:
gmm_dist = snsims.GMM_SALT2Params(numSN=len(zdist.zSamples), 
                                  zSamples=zdist.zSamples, 
                                  rng=np.random.RandomState(8),
                                  cosmo=DC2,
                                  mjdmin=59580., surveyDuration=10)

In [ ]:
df = gmm_dist.paramSamples

In [ ]:
df.z.hist(bins=np.arange(0., 1.4, 0.04))

In [ ]:
df.x1.hist()

In [ ]:
rng = np.random.RandomState(8)

In [ ]:
df['t0'] = np.random.uniform(59580, 59580 + 3650., size=len(df))
df['snid'] = df.index.values
df.set_index('snid', inplace=True)

In [ ]:
df.to_csv('sntable.csv')

In [ ]:
dflowz = df.query('z < 0.95')

In [ ]:
hosted = dflowz.sample(frac=0.9)

In [ ]:
binwidth = 0.01
hosted['zbin'] = hosted.z//binwidth
g['zbin'] = g.redshift//binwidth
g.zbin = g.zbin.astype(np.int)
hosted.zbin = hosted.zbin.astype(np.int)

In [ ]:
zbins = hosted.zbin.unique()

In [ ]:
g.columns

In [ ]:
hosted.head()

In [ ]:
syslist = []

for zbin in hosted.zbin.unique():
    querystring = 'zbin==@zbin'
    hostedtmp = hosted.query(querystring)
    numSN = hostedtmp.z.size
    gtmp  = g.query(querystring)
    tot = gtmp.totalMassStellar.sum()
    gids = np.random.choice(gtmp.galaxy_id, size=numSN, replace=False, p=gtmp.totalMassStellar/tot)
    print(len(hostedtmp), len(gids))
    syslist.append(pd.DataFrame(dict(snid=hostedtmp.reset_index().snid, galaxy_id=gids)))
    joiner = pd.concat(syslist).set_index('snid')
    return joiner

In [ ]:
joiner = pd.concat(syslist).set_index('snid')

In [ ]:
joiner.head()

In [ ]:
h = hosted.join(joiner)

In [ ]:
h.head()

In [ ]:
any(g.galaxy_id.isnull())

In [ ]:
h.galaxy_id.unique().size -  h.galaxy_id.size

In [ ]:
joiner.loc[17905]

In [ ]:
joined = h.set_index('galaxy_id').join(g.set_index('galaxy_id'), rsuffix='_gal')

In [ ]:
X = joined.z - joined.redshift

In [ ]:
fig, ax = plt.subplots()
_ = joined.totalMassStellar.apply(np.log10).hist(ax =ax, histtype='step', normed=1, color='k')
_ = g.totalMassStellar.apply(np.log10).hist(ax =ax, histtype='step', normed=1, color='r')

In [ ]:
galprops = g.set_index('galaxy_id').join(joiner.set_index('galaxy_id'))

In [ ]:
joined.columns

In [ ]:
for x in ['diskMassStellar_catsim', 'zbin_gal', 'uniqueId', 'totalMassStellar_catsim']:
    del joined[x]

In [ ]:
joined.columns

In [ ]:
joined.rename(columns=dict(raJ2000='raHost', decJ2000='decHost'), inplace=True)

In [ ]:
joined[['c', u'mB', u'x0', u'x1', u't0', u'zbin', u'diskMassStellar',
       u'morphology/diskMinorAxisArcsec',
       u'morphology/spheroidMinorAxisArcsec', u'totalMassStellar', u'raHost', u'decHost',
       u'redshift']].to_csv('SNWithHost')
joiner.to_csv('galaxy_SN_map.csv')

In [ ]:
joined['random'] = np.random.Unif

# Scratch

In [ ]:
hosted.z.size

In [ ]:
import pandas as pd

In [ ]:
gals = pd.read_csv('~/data/LSST/DC2/protDC2.csv.gz')

In [ ]:
g = gals.query('redshift < 0.95')

In [ ]:
df.head()